
![](https://www.tjsp.jus.br/Content/GaleriaExPresidentes/palacio.jpg)

## Tribunal de Justiça do Estado de São Paulo.
O Tribunal de Justiça do Estado de São Paulo é considerado o maior Tribunal do mundo em volume de processos. O número de ações demandadas no judiciário estadual paulista corresponde a 26% do total de processos em andamento em toda a Justiça brasileira, incluindo cortes federais e tribunais superiores (dados do relatório “Justiça em Números 2018”, produzido pelo Conselho Nacional de Justiça). Consequentemente, é o tribunal com a maior força de trabalho: 2,6 mil magistrados e aproximadamente 43 mil servidores, em 319 comarcas do Estado.


* [<font size=3>Carregando e limpando os dados</font>](#1)
* [<font size=3>Visualizações </font>](#2)
* [<font size=3>Referências </font>](#3)

## Carregando e limpando os dados <a id="1"></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid', palette='muted', font_scale=1.5)

In [ ]:
df = pd.read_csv('../input/salaries-of-magistrates-brasil/contracheque.csv')
df.head()

In [ ]:
df.info()

Porcentagem de arquivos faltantes

In [ ]:
df.isnull().mean() * 100

Como os dados não estão dividos por estado, vou selecionar somente os dados do Tribunal de Justiça do Estado de São Paulo

In [ ]:
tribunal_sp = df[df['orgao'] == 'Tribunal de Justiça do Estado de São Paulo'].drop('cpf', axis=1)
tribunal_sp.describe()

In [ ]:
tribunal_sp.describe().columns.to_list()
cargos = list(tribunal_sp['cargo'].unique())

## Visualizações <a id="2"></a>

In [ ]:
fig, axes = plt.subplots(figsize=(18,9))
tribunal_sp.groupby(['mesano_de_referencia'])[tribunal_sp.describe().columns.to_list()].sum().plot(kind='bar', rot=45, ax=axes)
axes.set_yticklabels(['${:,}'.format(int(x)) for x in axes.get_yticks().tolist()])
axes.legend(tribunal_sp.describe().columns.to_list(), bbox_to_anchor=(1, 1), title='Salários e Benefícios')
axes.set(Title='Gastos Mensais Salários e Benefícios de Juízes', xlabel='Mês');

Selecionar somente as colunas que são de benefícios e indenizações.

In [ ]:
cols  = ['subsidio',
 'direitos_pessoais',
 'indenizacoes',
 'direitos_eventuais',
 'previdencia_publica',
 'imposto_de_renda',
 'descontos_diversos',
 'retencao_por_teto_constitucional',
 'diarias']

fig, axes = plt.subplots(figsize=(18,9))
tribunal_sp.groupby(['mesano_de_referencia'])[cols].sum().plot(kind='bar', rot=45, ax=axes)
axes.set_yticklabels(['${:,}'.format(int(x)) for x in axes.get_yticks().tolist()])
axes.legend(tribunal_sp.describe().columns.to_list(), bbox_to_anchor=(1, 1), title='Benefícios e indenizações')
axes.set(Title='Gastos Mensais Benefícios e Indenizações de Juízes', xlabel='Mês');

In [ ]:
fig, axes = plt.subplots(figsize=(15,6))
tribunal_sp['cargo'].value_counts(normalize=True, ascending=True).plot(kind='barh', cmap='bone', ax=axes,
                                                                  title='Cargos');

In [ ]:
def plot_mes(col, df, title):
    fig, ax = plt.subplots(figsize=(18,6))
    df.groupby(['mesano_de_referencia'])[col].sum().plot(rot=45, kind='bar', ax=ax, legend=True, cmap='bone')
    ax.set_yticklabels(['${:,}'.format(int(x)) for x in ax.get_yticks().tolist()])
    ax.set(Title=title, xlabel='Mês')
    return ax

In [ ]:
plot_mes('imposto_de_renda', tribunal_sp, 'Retenção Imposto de Renda dos Juízes');

In [ ]:
plot_mes('rendimento_liquido', tribunal_sp, 'Gasto mensais salários do Juízes');

In [ ]:
plot_mes('diarias', tribunal_sp, 'Gasto mensais diárias');

In [ ]:
def plot_cargo_sal(cargos, col, title):
    for x in cargos:
        cargo = tribunal_sp[tribunal_sp['cargo']==x]
        ax = cargo.groupby(['mesano_de_referencia'])[col].mean().plot(figsize=(20,8), linewidth=3, rot=45)
        ax.set_yticklabels(['${:,}'.format(int(x)) for x in ax.get_yticks().tolist()])
        ax.legend(cargos, bbox_to_anchor=(1, 1), title='Cargos')
        ax.set(Title=title, xlabel='Mês')
    return ax

In [ ]:
plot_cargo_sal(cargos, 'rendimento_liquido', 'Média de salário por cargo');

Podemos ver que os Juízes, ganham em média acima do teto constitucional que é R$ 39.293.

In [ ]:
plot_cargo_sal(cargos, 'subsidio', 'Média de subsídios por cargo');

In [ ]:
plot_cargo_sal(cargos, 'total_de_rendimentos', 'Média de salário bruto por cargo');

Selecionamos os dez tribunais com os maiores gastos com salários dos juízes.

In [ ]:
mask = list(df.groupby(['orgao'])['total_de_rendimentos'].sum().sort_values(ascending=False)[:10].index)
tribunais = df[df['orgao'].isin(mask)]
ax = tribunais.groupby(['mesano_de_referencia', 'orgao'])['rendimento_liquido'].sum().unstack().plot(kind='bar', figsize=(18,8), cmap='bone', rot=45)
ax.set_yticklabels(['${:,}'.format(int(x)) for x in ax.get_yticks().tolist()])
ax.legend(bbox_to_anchor=(1, 1), title='')
ax.set(Title='Gasto com pagamentos por mês', xlabel='Mês');

## Referências <a id='3'></a>
[Quem somos? tjsp](https://www.tjsp.jus.br/QuemSomos)<br>
[O Judiciário Mais Caro do Mundo](https://jus.com.br/artigos/66373/o-judiciario-mais-caro-do-mundo)<br>
[Magistrados continuam ultrapassando o teto constitucional](https://blogs.correiobraziliense.com.br/servidor/magistrados-continuam-ultrapassando-o-teto-constitucional/)